In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader


In [ ]:

df_nichenet = pd.read_csv("/Users/Kathyayini R/Documents/Dissertation/P057_singleCellCrosstalks/P057_singleCellCrosstalks/Data/colon_zhang/nichenet_result_runs/TAMs_to_Global/communication_network.csv")   # NicheNet subcluster->macro
df_sub = pd.read_csv("/Users/Kathyayini R/Documents/Dissertation/P057_singleCellCrosstalks/P057_singleCellCrosstalks/Data/colon_zhang/Subcluster_Communication_Analysis/Full_Communication_Network.csv")   # CellChat subcluster->subcluster
df_global = pd.read_csv("/Users/Kathyayini R/Documents/Dissertation/P057_singleCellCrosstalks/P057_singleCellCrosstalks/Data/colon_zhang/GlobalCluster_Communication_Analysis/Full_Communication_Network.csv") # CellChat global->macro



#Preprocess NicheNet
df_nichenet = df_nichenet.rename(columns={"source": "sender", "target": "receiver_macro"})
df_nichenet["nn_present"] = 1
df_nichenet["nn_rank"] = df_nichenet.groupby("sender").cumcount() + 1
df_nichenet["nn_rank"] = df_nichenet["nn_rank"] / df_nichenet["nn_rank"].max()


#Preprocess CellChat subcluster

df_sub = df_sub.rename(columns={"source": "sender", "target": "receiver_sub"})
df_sub["cc_prob_sub"] = df_sub["prob"] if "prob" in df_sub.columns else 0.0
df_sub["cc_pval_sub"] = df_sub["pval"] if "pval" in df_sub.columns else 1.0
df_sub = df_sub[["sender", "receiver_sub", "ligand", "receptor", "cc_prob_sub", "cc_pval_sub"]]


In [ ]:

#Preprocess CellChat global
if "target" in df_global.columns:
    df_global = df_global.rename(columns={"source": "sender_macro", "target": "receiver_macro"})
else:
    df_global = df_global.rename(columns={"source": "sender_macro"})

if "prob" in df_global.columns:
    df_global["cc_prob_macro"] = df_global["prob"]
if "pval" in df_global.columns:
    df_global["cc_pval_macro"] = df_global["pval"]

keep_cols = [c for c in ["sender_macro","receiver_macro","ligand","receptor","cc_prob_macro","cc_pval_macro"] if c in df_global.columns]
df_global = df_global[keep_cols]


In [ ]:

#Map subclusters -> macros
def map_to_macro(cell_id):
    if pd.isna(cell_id): return "Other"
    if "Epi" in cell_id: return "Epithelial cell"
    if "Fibro" in cell_id: return "Fibroblast"
    if "Endo" in cell_id: return "Endothelial"
    if "CD4" in cell_id: return "CD4 T cell"
    if "CD8" in cell_id: return "CD8 T cell"
    if "NK" in cell_id or "ILC" in cell_id: return "ILC"
    if "B" in cell_id: return "B cell"
    if any(x in cell_id for x in ["Macro","TAM","Mono","cDC","pDC"]): return "Myeloid cell"
    if "Malig" in cell_id: return "Malignant cell"
    return "Other"

df_sub["receiver_macro"] = df_sub["receiver_sub"].apply(map_to_macro)


In [ ]:

merged = pd.merge(df_sub, df_nichenet, 
                  on=["sender","ligand","receptor","receiver_macro"], 
                  how="outer", suffixes=("_sub","_nn"))

if "receiver_macro" in df_global.columns:
    merged = pd.merge(merged, df_global, on=["ligand","receptor","receiver_macro"], how="left")


tmp_macro = merged["sender"].apply(map_to_macro)
merged["is_autocrine"] = (tmp_macro == merged["receiver_macro"]).astype(int)
merged["is_multisubunit"] = merged["receptor"].astype(str).str.contains("_").astype(int)
redundant_ligands = merged.groupby("ligand")["receptor"].nunique()
merged["receptor_redundancy"] = merged["ligand"].map(lambda x: 1 if redundant_ligands.get(x,0)>1 else 0)


merged = merged.fillna({
    "nn_present": 0, "nn_rank": 1.0,
    "cc_prob_sub": 0.0, "cc_pval_sub": 1.0,
    "cc_prob_macro": 0.0, "cc_pval_macro": 1.0
})


#ML: XGBoost baseline 

features = ["cc_prob_sub","cc_pval_sub","cc_prob_macro","cc_pval_macro",
            "is_autocrine","is_multisubunit","receptor_redundancy"]

X = merged[features].values.astype("float32")
y = merged["nn_present"].astype("float32").values

cv = GroupKFold(n_splits=5)
y_true, y_pred = [], []
for train_idx, test_idx in cv.split(X,y,groups=merged["ligand"]):
    model = XGBClassifier(objective="binary:logistic", eval_metric="logloss")
    model.fit(X[train_idx], y[train_idx])
    preds = model.predict_proba(X[test_idx])[:,1]
    y_true.extend(y[test_idx])
    y_pred.extend(preds)

print("XGB ROC-AUC:", roc_auc_score(y_true,y_pred))
print("XGB PR-AUC:", average_precision_score(y_true,y_pred))

final_model = XGBClassifier(objective="binary:logistic", eval_metric="logloss")
final_model.fit(X,y)
merged["xgb_prob"] = final_model.predict_proba(X)[:,1]


In [ ]:

#DL: Neural Net + embeddings

X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)
dataset = TensorDataset(X_tensor,y_tensor)
loader = DataLoader(dataset,batch_size=128,shuffle=True)

class EdgeNet(nn.Module):
    def __init__(self,in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim,64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64,32),  # embedding layer
            nn.ReLU(),
            nn.Linear(32,1),
            nn.Sigmoid()
        )
    def forward(self,x): return self.net(x)

model = EdgeNet(X.shape[1])
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(20):
    for xb,yb in loader:
        preds = model(xb).squeeze()
        loss = loss_fn(preds,yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1)%5==0:
        print(f"Epoch {epoch+1}, Loss={loss.item():.4f}")

with torch.no_grad():
    merged["dl_prob"] = model(X_tensor).squeeze().numpy()

# Extract embeddings
extractor = nn.Sequential(*list(model.net.children())[:-2])
with torch.no_grad():
    embeddings = extractor(X_tensor).numpy()
for i in range(embeddings.shape[1]):
    merged[f"emb_{i}"] = embeddings[:,i]


In [ ]:
receiver_emb = merged.groupby("receiver_macro")[[f"emb_{i}" for i in range(embeddings.shape[1])]].mean()
kmeans = KMeans(n_clusters=3,random_state=42).fit(receiver_emb)
receiver_emb["module"] = kmeans.labels_
print(receiver_emb.head())


In [ ]:

#Visualization

# t-SNE of receiver embeddings
tsne = TSNE(n_components=2,random_state=42, perplexity=3)
tsne_coords = tsne.fit_transform(receiver_emb[[f"emb_{i}" for i in range(embeddings.shape[1])]])

plt.figure(figsize=(7,6))
sns.scatterplot(x=tsne_coords[:,0], y=tsne_coords[:,1],
                hue=receiver_emb["module"], palette="Set2", s=120)
for i, txt in enumerate(receiver_emb.index):
    plt.text(tsne_coords[i,0]+0.1, tsne_coords[i,1], txt, fontsize=8)
plt.title("Receiver modules learned from DL embeddings")
plt.show()

# Heatmap of embeddings
plt.figure(figsize=(8,6))
sns.heatmap(receiver_emb.drop("module",axis=1), cmap="viridis")
plt.title("Receiver latent embeddings (DL)")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Filter only TAM senders
tam_edges = merged[merged["sender"].isin(["hM12_TAM-C1QC", "hM13_TAM-SPP1"])]

#Aggregate by sender + macro
macro_strength = (
    tam_edges.groupby(["sender", "receiver_macro"])["dl_prob"]
    .mean()
    .reset_index()
)

#Normalize to % per sender (use transform for alignment!)
macro_strength["percent"] = (
    macro_strength.groupby("sender")["dl_prob"]
    .transform(lambda x: 100 * x / x.sum())
)

print(macro_strength.sort_values(["sender", "percent"], ascending=[True, False]))

# Stacked barplot
plt.figure(figsize=(10,6))
sns.barplot(
    data=macro_strength,
    x="sender", y="percent", hue="receiver_macro", palette="tab10"
)

plt.ylabel("Share of total interaction (%)")
plt.xlabel("TAM Program")
plt.title("Relative targeting of receiver macros by C1QC vs SPP1 TAMs")
plt.legend(title="Receiver Macro", bbox_to_anchor=(1.05,1), loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

immune = ["B cell", "CD4 T cell", "CD8 T cell", "Myeloid cell", "ILC"]
stromal = ["Epithelial cell", "Fibroblast", "Malignant cell"]

def categorize(macro):
    if macro in immune: return "Immune"
    elif macro in stromal: return "Stromal"
    else: return "Other"

macro_strength["category"] = macro_strength["receiver_macro"].apply(categorize)
collapsed = (
    macro_strength.groupby(["sender","category"])["percent"]
    .sum()
    .reset_index()
)


receiver_emb = merged.groupby("receiver_sub")[["dl_prob"]].mean().reset_index()


emb_cols = [c for c in merged.columns if c.startswith("emb_")]
receiver_latent = merged.groupby("receiver_sub")[emb_cols].mean().reset_index()

#Run t-SNE on embeddings
tsne = TSNE(n_components=2, random_state=42, perplexity=10)
coords = tsne.fit_transform(receiver_latent[emb_cols])
receiver_latent["tSNE1"] = coords[:,0]
receiver_latent["tSNE2"] = coords[:,1]

#Assign macro categories to subclusters
macro_map = merged[["receiver_sub","receiver_macro"]].drop_duplicates()
receiver_latent = receiver_latent.merge(macro_map, on="receiver_sub", how="left")

fig, axes = plt.subplots(1,2, figsize=(14,6))

# (a) Stacked bar: Macro-level immune vs stromal
sns.barplot(
    data=collapsed, x="sender", y="percent", hue="category", 
    palette="Set2", ax=axes[0]
)
axes[0].set_ylabel("Share of total interaction (%)")
axes[0].set_title("Macro-level: Immune vs Stromal balance")

# (b) Subcluster t-SNE: immune vs stromal receivers
sns.scatterplot(
    data=receiver_latent, x="tSNE1", y="tSNE2", 
    hue="receiver_macro", palette="tab10", ax=axes[1], s=70
)
for i,row in receiver_latent.iterrows():
    axes[1].text(row.tSNE1, row.tSNE2, row["receiver_sub"], fontsize=7)

axes[1].set_title("Subcluster-level: fine-grained targeting")
axes[1].legend(bbox_to_anchor=(1.05,1), loc="upper left")

plt.tight_layout()
plt.show()

In [ ]:

#Compute average interaction strength for each TAM→macro
macro_targets = (
    merged.groupby(["sender","receiver_macro"])["dl_prob"]
    .mean()
    .reset_index()
)

#Pivot to get two columns: C1QC and SPP1
macro_pivot = macro_targets.pivot(
    index="receiver_macro", columns="sender", values="dl_prob"
).fillna(0)

#Merge with t-SNE embeddings
plot_df = receiver_latent.merge(
    macro_pivot, left_on="receiver_macro", right_index=True, how="left"
)

#Plot: bubble size = C1QC, color = SPP1
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=plot_df,
    x="tSNE1", y="tSNE2",
    size="hM12_TAM-C1QC", sizes=(50,600),
    hue="hM13_TAM-SPP1", palette="coolwarm", legend="brief"
)


for i, txt in enumerate(plot_df["receiver_macro"]):
    plt.text(plot_df["tSNE1"].iloc[i]+0.2, plot_df["tSNE2"].iloc[i], txt, fontsize=8)

plt.title("C1QC vs SPP1 TAM targeting on receiver macros")
plt.xlabel("tSNE1")
plt.ylabel("tSNE2")
plt.legend(bbox_to_anchor=(1.05,1), loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

pivot = macro_targets.pivot(index="receiver_macro", columns="sender", values="dl_prob").fillna(0)


pivot = pivot[["hM12_TAM-C1QC", "hM13_TAM-SPP1"]]

#Δ score: SPP1 - C1QC
pivot["delta"] = pivot["hM13_TAM-SPP1"] - pivot["hM12_TAM-C1QC"]


delta_df = receiver_latent.merge(pivot["delta"], left_on="receiver_macro", right_index=True, how="left")


#Plot Δ-scores on t-SNE

plt.figure(figsize=(8,6))
sc = plt.scatter(
    delta_df["tSNE1"], delta_df["tSNE2"],
    c=delta_df["delta"], cmap="bwr", s=150, edgecolor="k"
)

for i, txt in enumerate(delta_df["receiver_macro"]):
    plt.text(delta_df["tSNE1"].iloc[i]+0.3, delta_df["tSNE2"].iloc[i], txt, fontsize=8)

plt.colorbar(sc, label="Δ score (SPP1 - C1QC)")
plt.title("Immune vs Stromal preference of TAM programs (Δ-scores)")
plt.xlabel("tSNE1")
plt.ylabel("tSNE2")
plt.tight_layout()
plt.show()



In [ ]:
top_delta = pivot["delta"].sort_values(ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x=top_delta.values, y=top_delta.index, palette="coolwarm")
plt.axvline(0, color="k", linestyle="--")
plt.xlabel("Δ score (SPP1 - C1QC)")
plt.ylabel("Receiver Macro")
plt.title("Top receiver macros biased to SPP1 vs C1QC TAMs")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


#pivot so each receiver has one row, columns = senders
pivot1 = macro_targets.pivot(index="receiver_macro", columns="sender", values="dl_prob").fillna(0)


pivot1 = pivot1[["hM12_TAM-C1QC", "hM13_TAM-SPP1"]]

#Δ score: SPP1 - C1QC
pivot1["delta"] = pivot1["hM12_TAM-C1QC"] - pivot1["hM13_TAM-SPP1"]


delta_df1 = receiver_latent.merge(pivot1["delta"], left_on="receiver_macro", right_index=True, how="left")


plt.figure(figsize=(8,6))
sc = plt.scatter(
    delta_df["tSNE1"], delta_df["tSNE2"],
    c=delta_df["delta"], cmap="bwr", s=150, edgecolor="k"
)

for i, txt in enumerate(delta_df["receiver_macro"]):
    plt.text(delta_df1["tSNE1"].iloc[i]+0.3, delta_df1["tSNE2"].iloc[i], txt, fontsize=8)

plt.colorbar(sc, label="Δ score (C1QC - SPP1)")
plt.title("Immune vs Stromal preference of TAM programs (Δ-scores)")
plt.xlabel("tSNE1")
plt.ylabel("tSNE2")
plt.tight_layout()
plt.show()




In [ ]:
top_delta1 = pivot1["delta"].sort_values(ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x=top_delta1.values, y=top_delta1.index, palette="coolwarm")
plt.axvline(0, color="k", linestyle="--")
plt.xlabel("Δ score (C1QC - SPP1)")
plt.ylabel("Receiver Macro")
plt.title("Top receiver macros biased to  C1QC TAMs")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter for C1QC sender
c1qc_edges = merged[merged["sender"] == "hM12_TAM-C1QC"]

# Group by receiver macro
c1qc_targets = (
    c1qc_edges.groupby("receiver_macro")["dl_prob"]
    .mean()
    .reset_index()
    .sort_values("dl_prob", ascending=False)
)

print("C1QC-TAM targeting (macro level):")
print(c1qc_targets)

# --- Barplot ---
plt.figure(figsize=(8,5))
sns.barplot(data=c1qc_targets, x="dl_prob", y="receiver_macro", palette="viridis")
plt.xlabel("Average fused probability (DL)")
plt.ylabel("Receiver Macro")
plt.title("C1QC-TAM preferentially targets immune macros")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter for SPP1 sender
c1qc_edges = merged[merged["sender"] == "hM13_TAM-SPP1"]

# Group by receiver macro
c1qc_targets = (
    c1qc_edges.groupby("receiver_macro")["dl_prob"]
    .mean()
    .reset_index()
    .sort_values("dl_prob", ascending=False)
)

print("hM13_TAM-SPP1 targeting (macro level):")
print(c1qc_targets)

# --- Barplot ---
plt.figure(figsize=(8,5))
sns.barplot(data=c1qc_targets, x="dl_prob", y="receiver_macro", palette="viridis")
plt.xlabel("Average fused probability (DL)")
plt.ylabel("Receiver Macro")
plt.title("hM13_TAM-SPP1 preferentially targets immune macros")
plt.tight_layout()
plt.show()


In [ ]:
from adjustText import adjust_text

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_coords = tsne.fit_transform(receiver_sub_emb[[f"emb_{i}" for i in range(embeddings.shape[1])]])

plt.figure(figsize=(12,9))
sns.scatterplot(
    x=tsne_coords[:,0], y=tsne_coords[:,1],
    hue=receiver_sub_emb["module"], palette="tab10", s=60, alpha=0.8
)


receiver_labels = [idx.split("_",1)[-1] for idx in receiver_sub_emb.index]


texts = []
for i, txt in enumerate(receiver_labels):
    texts.append(plt.text(tsne_coords[i,0], tsne_coords[i,1], txt, fontsize=7))
adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray', lw=0.5))

plt.title("Receiver subcluster modules (DL embeddings)")
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")
plt.legend(title="Module", bbox_to_anchor=(1.05,1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
#sybclusters
sub_edges = merged[merged["sender"].isin(["hM12_TAM-C1QC", "hM13_TAM-SPP1"])]

sub_strength = (
    sub_edges.groupby(["sender", "receiver_sub"])["dl_prob"]
    .mean()
    .reset_index()
)


sub_strength["percent"] = (
    sub_strength.groupby("sender")["dl_prob"]
    .transform(lambda x: 100 * x / x.sum())
)

pivot_sub = sub_strength.pivot(
    index="receiver_sub", columns="sender", values="dl_prob"
).fillna(0)


for col in ["hM12_TAM-C1QC", "hM13_TAM-SPP1"]:
    if col not in pivot_sub.columns:
        pivot_sub[col] = 0

pivot_sub["delta"] = pivot_sub["hM13_TAM-SPP1"] - pivot_sub["hM12_TAM-C1QC"]
pivot_sub = pivot_sub.sort_values("delta", ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=pivot_sub["delta"], y=pivot_sub.index, palette="coolwarm")
plt.axvline(0, color="k", linestyle="--")
plt.xlabel("Δ score (SPP1 - C1QC)")
plt.ylabel("Receiver Subcluster")
plt.title("Subcluster bias of SPP1 vs C1QC TAMs")
plt.tight_layout()
plt.show()

In [ ]:
top_sub = (
    sub_strength.groupby("sender")
    .apply(lambda g: g.nlargest(10, "percent"))
    .reset_index(drop=True)
)
top_sub["receiver_sub"] = top_sub["receiver_sub"].astype("category")
plt.figure(figsize=(10,6))
sns.barplot(
    data=top_sub,
    x="sender", y="percent", hue="receiver_sub", palette="tab20"
)
plt.ylabel("Share of total interaction (%)")
plt.xlabel("TAM subcluster")
plt.title("Top 10 receiver subclusters targeted by C1QC vs SPP1 TAMs")
plt.legend(title="Receiver Subcluster", bbox_to_anchor=(1.05,1), loc="upper left")
plt.tight_layout()
plt.show()
